## STEP 0. Configure runtime to work with GPU

Go to **> Runtime > Change Runtime Type** And select **GPU** From the **Hardware accelerator** drop down menu

## STEP 1. Connect the Colab notebook to Google Drive

Now we're going to mount your Google Drive folder to the local Colab VM. This step is required in all notebooks and will require your manual interaction.

* Execute the following cell _(Click on Play button or press CTRL + ENTER)_ and click on the link to authorize your notebook to access to your Google Drive. 
* Paste the code Google will give to you and push `enter`

In [ ]:
# This cell imports the drive library and mounts your Google Drive as a VM local drive. 
# You can access to your Drive files using this path "/content/gdrive/My Drive/"

from google.colab import drive
drive.mount('/content/gdrive')

## STEP 2. Check CUDA release version

Nvidia CUDA is pre-installed on Colab notebooks. Now we'll check the version installed.

> _**BE AWARE:** In the future, the CUDA version will be upgraded on Colab and you should download another version of the cuDNN in the next step. Now is release 10.1 and we are using cuDNN (cudnn-10.1-linux-x64-v7.6.4.38.tgz) accordingly_

In [ ]:
# CUDA: Let's check that Nvidia CUDA is already pre-installed and which version is it.
!/usr/local/cuda/bin/nvcc --version


## STEP 3. Install cuDNN according to the current CUDA version
If you haven't already, you'll need to download cuDNN from Nvidia web site and add it to the cuDNN folder in the yolo_colab repository.

* Download cuDNN from [Nvidia website](https://developer.nvidia.com/cudnn) 

  * Right now, because we have _**CUDA 10.1**_ preinstalled in Colab runtime, you need download [cuDNN v7.6.4 for CUDA v10.1](https://developer.nvidia.com/rdp/cudnn-archive#a-collapse764-101) - the file is cudnn-10.1-linux-x64-v7.6.4.38

* Drag that _**tgz**_ file into your _**yolo_colab/cuDNN**_ folder on Google Drive.



In [ ]:
# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
# Update the file name here if you have a newer version
!tar -xzvf /content/gdrive/My\ Drive/yolo_colab/cuDNN/cudnn-10.1-linux-x64-v7.6.4.38.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

## STEP 4. Cloning and compiling Darkent. 

In this step we'll clone the darknet repo and compile it. 
* Clone Repo
* Compile Darknet
* Copy compiled version to Drive

In [ ]:
# Leave this code uncommented on the very first run of your notebook or if you ever need to recompile darknet again.
# Comment this code on the future runs.
!git clone https://github.com/kriyeng/darknet/
%cd darknet

# I have a branch where I have done the changes commented above
!git checkout feature/google-colab

#Compile Darknet
!make

#Copies the Darknet compiled version to Google drive
!cp ./darknet /content/gdrive/My\ Drive/yolo_colab/bin/darknet/

## STEP 5. Copying files from  Google Drive

Here we will just copy down all the files needed for training from our yolo_colab folder on Google Drive.

- `yolo_colab/cfg/` contains config files
- `yolo_colab/img/` contains training images (`.jpg`) and bounding box (`.txt`) files


In [ ]:
# Makes a dir for yolo_colab and move there
%%capture
!mkdir yolo_colab
%cd yolo_colab

# Copy the Darknet compiled version to the VM local drive
!cp /content/gdrive/My\ Drive/yolo_colab/bin/darknet ./darknet

# Set execution permissions to Darknet
!chmod +x ./darknet

# copy yolov3 config files and images to VM local drive
!cp /content/gdrive/My\ Drive/yolo_colab/cfg/* .
!tar -xzvf /content/gdrive/My\ Drive/yolo_colab/img.tar.gz

## STEP 6. Download initial weights

Finally, we want to start training with some pretrained weights to jumpstart our training. We download these from the YOLO author's [website](https://pjreddie.com/darknet/).


In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

## Step 7. Train your model!

When you execute the following command, your model will start training.

You will have a log line per epoch. On each iteration you will see how your training is going.

This cell should run for several hours until your session on Google Colab times out or you quit the program.  

Darknet copies a backup of your trained weights every 100 iterations. The `obj.data` file sets the trained weights to be stored in the `yolo_colab/backup/yolov3_last.weights` folder on Google Drive. This will be the output model weights file that's used ot detect objects.

> **When this script is done, you can test the last output model weights using the `DarknetYOLOTest.ipynb` notebook.**

In [ ]:
!./darknet detector train "obj.data" "yolov3.cfg" "darknet53.conv.74" -dont_show 

## Credits
Props to [David Ibáñez](https://twitter.com/dav_ibanez) for the original basis for this example: [yolo-on-colab-notebook](https://github.com/kriyeng/yolo-on-colab-notebook).

The image files come from Google [OpenImages](https://storage.googleapis.com/openimages/web/index.html).